In [1]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

from torchvision.models import resnet18
from torchvision import datasets, transforms

from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

import os
import random
from tqdm import tqdm
from PIL import Image

In [2]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

In [3]:
print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

# CPU
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
cpu MHz		: 2000.192
cpu cores	: 2


In [4]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(7)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)


In [8]:
class JpegCompression(transforms.Lambda):
    def __init__(self, quality_lower=60, quality_upper=100, p=0.5):
        super().__init__(self.apply_jpeg_compression)
        self.quality_lower = quality_lower
        self.quality_upper = quality_upper
        self.probability = p

    def apply_jpeg_compression(self, img):
        if random.random() < self.probability:
            quality = random.randint(self.quality_lower, self.quality_upper)
            buffer = io.BytesIO()
            img.save(buffer, format="JPEG", quality=quality)
            buffer.seek(0)
            img = Image.open(buffer)
        return img

In [9]:
def random_color_jitter():
    return transforms.ColorJitter(
        brightness=random.uniform(0.1, 0.3),
        contrast=random.uniform(0.1, 0.3),
        saturation=random.uniform(0.1, 0.3)
    )

In [10]:
import torchvision.transforms as transforms
import io
from torchvision.transforms.functional import to_pil_image, to_tensor 

# 필요한 크기
target_size = 380  # 원하는 크기를 256x256으로 설정


transform = transforms.Compose([
    transforms.Resize(target_size, interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.CenterCrop(target_size),
    JpegCompression(quality_lower=60, quality_upper=90, p=0.2),
    transforms.RandomApply([transforms.RandomRotation(90)], p=0.2), #20% 만큼만 90도 각도로 파일변환
    transforms.RandomHorizontalFlip(p=0.2), # 75.35%
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
dataset = datasets.ImageFolder(root='/kaggle/input/image-classification-2024-spring/dataset/train', transform=transform)

In [12]:
dataset_size = len(dataset)
train_size = int(dataset_size * 0.95)
val_size = dataset_size - train_size

trainset, valset = random_split(dataset, [train_size, val_size])

In [13]:
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(valset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

In [14]:
import torchvision.models as models
# EfficientNet V2-S 모델을 불러옵니다. 미리 학습된 가중치를 사용합니다.
model = models.efficientnet_v2_s(pretrained=True)

num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)

model = nn.DataParallel(model, device_ids=[0, 1])
model = model.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 167MB/s]


In [15]:
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR

steps_per_epoch = len(train_loader)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
# OneCycleLR 스케줄러 설정
scheduler = OneCycleLR(
    optimizer,
    max_lr=1e-3,             # 최대 학습률
    epochs=7,                # 총 학습 에폭 수
    steps_per_epoch=steps_per_epoch,
    pct_start=0.3,           # 학습률이 최대가 되기까지의 비율
    anneal_strategy='cos',   # 코사인 곡선으로 학습률 감소
    div_factor=25.0,         # 초기 학습률은 max_lr / div_factor
    final_div_factor=1e4     # 최종 학습률은 max_lr / final_div_factor
)
scaler = GradScaler()

In [16]:
if not os.path.exists('checkpoint'):
    os.makedirs('checkpoint')

best_acc = 0.

In [17]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())


In [18]:
print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

# CPU
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
cpu MHz		: 2000.192
cpu cores	: 2


In [19]:
for epoch in range(7):
    model.train()
    running_loss = 0.0
    preds = []
    labels = []

    for inputs, label in tqdm(train_loader):
        inputs = inputs.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, label.long())

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        preds += predicted.detach().cpu().numpy().tolist()
        labels += label.detach().cpu().numpy().tolist()
    train_accuracy = accuracy_score(labels, preds)
    print(f'train_accuracy: {train_accuracy}')

    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for inputs, label in tqdm(val_loader):
            inputs = inputs.to(device)
            label = label.to(device)

            with autocast():
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_preds += predicted.detach().cpu().numpy().tolist()
                val_labels += label.detach().cpu().numpy().tolist()

    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f'val_accuracy: {val_accuracy}')

    if val_accuracy > best_acc:
        best_acc = val_accuracy
        torch.save(model.state_dict(), f'checkpoint/model4.pth')
    scheduler.step()

100%|██████████| 297/297 [06:42<00:00,  1.36s/it]


train_accuracy: 0.9316315789473684


100%|██████████| 16/16 [00:21<00:00,  1.33s/it]


val_accuracy: 0.999


100%|██████████| 297/297 [06:01<00:00,  1.22s/it]


...

val_accuracy: 1.0


100%|██████████| 297/297 [05:57<00:00,  1.21s/it]


train_accuracy: 0.9987894736842106


100%|██████████| 16/16 [00:18<00:00,  1.18s/it]

val_accuracy: 1.0
